In [1]:
import wandb
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer
import sentencepiece
from transformers import XLNetForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch
from ray.air import session
import ray
from ray import tune
import os

In [2]:
df = pd.read_csv("./dataset/WELFake_Dataset.csv")
df = df.dropna()
df['text'] = df['title'] + " " + df['text']


In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [4]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', cache_dir='./model')

def preprocess(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=256,
        return_tensors="pt"
    )

train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
eval_dataset = Dataset.from_pandas(eval_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

In [5]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/45783 [00:00<?, ? examples/s]

Map:   0%|          | 0/11446 [00:00<?, ? examples/s]

Map:   0%|          | 0/14308 [00:00<?, ? examples/s]

In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Get the predicted class by using argmax (for multi-class classification)
    preds = np.argmax(predictions, axis=1)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="binary")  # Use 'micro', 'macro', or 'weighted' based on the task

    return {
        'accuracy': accuracy,
        'f1': f1,
    }

In [7]:
model = XLNetForSequenceClassification.from_pretrained(
    'xlnet/xlnet-base-cased',
    num_labels=2,
    problem_type="single_label_classification",
    cache_dir='./model'
)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [9]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return {'accuracy': accuracy_score(labels, predictions)}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics
# )

# trainer.train()

In [10]:
def train_fn(config, model, train_dataset, eval_dataset):

    try:
        trial_dir = session.get_trial_dir()  # 例如：~/ray_results/test/trial_xxx/
        output_dir = os.path.join(trial_dir, "results")
    except Exception as e:
        print(f"路径错误: {str(e)}")
        raise
    
    # Update training arguments with the hyperparameters from Ray Tune
    training_args = TrainingArguments(
        run_name = "ray_test",
        output_dir=output_dir,
        num_train_epochs=1,  
        
        per_device_train_batch_size=config["batch_size"],  # Hyperparameter from Ray Tune
        per_device_eval_batch_size=config["batch_size"],   # Hyperparameter from Ray Tune
        # warmup_steps=config["warmup_steps"],               # Hyperparameter from Ray Tune
        warmup_steps=500,
        learning_rate=config["learning_rate"],              # Hyperparameter from Ray Tune
        # learning_rate=1e-5,
        
        weight_decay=0.01,
        logging_dir=os.path.join(trial_dir, "logs"),  
        logging_steps=500,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        metric_for_best_model="eval_accuracy",
    )

    
    # Initialize the Trainer
    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=train_dataset, 
        eval_dataset=eval_dataset, 
        compute_metrics=compute_metrics,
    )
    try:
        # Train the model
        trainer.train()
    except Exception as e:
        print(f"训练失败: {str(e)}")
        raise

    try:
    # Evaluate the model
        eval_results = trainer.evaluate()
    except Exception as e:
        print(f"评估失败: {str(e)}")
        raise

    try:
    # Return the evaluation results to Ray Tune
        tune.report(metrics=eval_results)
        trainer.save_model(output_dir)
        tune.report(
            metrics=eval_results,
            checkpoint=tune.Checkpoint.from_directory(output_dir)  # 将模型目录作为检查点
        )
    except Exception as e:
        print(f"报告错误: {str(e)}")
        raise


In [11]:
search_space = {
    "learning_rate": tune.grid_search([1e-5, 2e-5]),
    "batch_size": tune.choice([8, 16]),
    # "warmup_steps": tune.choice([500, 1000, 2000]),
}

In [12]:
wandb.init(project="Mlops-fakenews", entity="yunchiz-new-york-university")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yunchiz (yunchiz-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [13]:
current_dir = os.getcwd()
storage_path = f"file://{current_dir}/ray_results"

train_fn_with_params = tune.with_parameters(train_fn, model=model, train_dataset=train_dataset, eval_dataset=eval_dataset)
ray.init(ignore_reinit_error=True)  # Initialize Ray
analysis = tune.run(
    train_fn_with_params,  # The training function that Ray Tune will use
    config=search_space,  # The search space of hyperparameters
    # resources_per_trial={"cpu": 1, "gpu": 1},
    resources_per_trial={"cpu": 0, "gpu": 1},
    num_samples=2,  # Number of trials (hyperparameter combinations)
    verbose=1,  # Verbosity level of Ray Tune
    storage_path=storage_path,
    name="ray_test",
)


(train_fn pid=22638) wandb: Currently logged in as: yunchiz (yunchiz-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(train_fn pid=22638) wandb: Tracking run with wandb version 0.19.9
(train_fn pid=22638) wandb: Run data is saved locally in /tmp/ray/session_2025-04-17_11-56-14_065850_20753/artifacts/2025-04-17_11-56-21/ray_test/working_dirs/train_fn_ecb7e_00000_0_batch_size=8,learning_rate=0.0000_2025-04-17_11-56-21/wandb/run-20250417_115632-lv2r0ig6
(train_fn pid=22638) wandb: Run `wandb offline` to turn off syncing.
(train_fn pid=22638) wandb: Syncing run ray_test
(train_fn pid=22638) wandb: ⭐️ View project at https://wandb.ai/yunchiz-new-york-university/huggingface
(train_fn pid=22638) wandb: 🚀 View run at https://wandb.ai/yunchiz-new-york-university/huggingface/runs/lv2r0ig6
  9%|▊         | 499/5723 [01:12<12:32,  6.94it/s]


(train_fn pid=22638) {'loss': 0.3187, 'grad_norm': 0.09295554459095001, 'learning_rate': 9.980000000000001e-06, 'epoch': 0.09}


 17%|█▋        | 1000/5723 [02:23<11:04,  7.11it/s]


(train_fn pid=22638) {'loss': 0.053, 'grad_norm': 0.0030184437055140734, 'learning_rate': 9.044610377177868e-06, 'epoch': 0.17}


 26%|██▌       | 1500/5723 [03:34<09:55,  7.10it/s]


...

100%|██████████| 2862/2862 [12:37<00:00,  4.41it/s]
(train_fn pid=24181) 
  0%|          | 0/716 [00:00<?, ?it/s]
(train_fn pid=24181) 
  0%|          | 3/716 [00:00<00:44, 15.94it/s]
(train_fn pid=24181) 
  1%|          | 5/716 [00:00<00:55, 12.84it/s]
(train_fn pid=24181) 
  1%|          | 7/716 [00:00<01:00, 11.77it/s]
(train_fn pid=24181) 
  1%|▏         | 9/716 [00:00<01:02, 11.25it/s]
(train_fn pid=24181) 
  2%|▏         | 11/716 [00:00<01:03, 11.05it/s]
(train_fn pid=24181) 
  2%|▏         | 13/716 [00:01<01:04, 10.88it/s]
(train_fn pid=24181) 
  2%|▏         | 15/716 [00:01<01:05, 10.68it/s]
(train_fn pid=24181) 
  2%|▏         | 17/716 [00:01<01:05, 10.66it/s]
(train_fn pid=24181) 
  3%|▎         | 19/716 [00:01<01:05, 10.66it/s]
(train_fn pid=24181) 
  3%|▎         | 21/716 [00:01<01:05, 10.58it/s]
(train_fn pid=24181) 
  3%|▎         | 23/716 [00:02<01:05, 10.64it/s]
(train_fn pid=24181) 
  3%|▎         | 25/716 [00:02<01:05, 10.60it/s]
(train_fn pid=24181) 
  4%|▍         |

(train_fn pid=24181) {'eval_loss': 0.016171641647815704, 'eval_accuracy': 0.9964179626070243, 'eval_f1': 0.9965333558806122, 'eval_runtime': 67.3889, 'eval_samples_per_second': 169.85, 'eval_steps_per_second': 10.625, 'epoch': 1.0}


(train_fn pid=24181) 
                                                   A
100%|██████████| 716/716 [01:07<00:00, 10.75it/s]
                                                 


(train_fn pid=24181) {'train_runtime': 829.3354, 'train_samples_per_second': 55.204, 'train_steps_per_second': 3.451, 'train_loss': 0.06189949124280928, 'epoch': 1.0}


100%|██████████| 716/716 [01:07<00:00, 10.57it/s]
(train_fn pid=24181) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/autodl-tmp/ray_results/ray_test/train_fn_ecb7e_00003_3_batch_size=16,learning_rate=0.0000_2025-04-17_11-56-21/checkpoint_000000)
2025-04-17 13:03:20,318	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/autodl-tmp/ray_results/ray_test' in 19.1601s.
2025-04-17 13:03:20,411	INFO tune.py:1041 -- Total run time: 4018.78 seconds (3999.45 seconds for the tuning loop).


In [14]:
best_trial = analysis.get_best_trial(metric="eval_accuracy", mode="max")

# 获取检查点路径（通过 checkpoint 属性）
best_checkpoint = best_trial.checkpoint
best_checkpoint_dir = best_checkpoint.to_directory()  # 提取检查点目录
print(f"最佳模型路径：{best_checkpoint_dir}")

# 加载模型
from transformers import AutoModel
best_model = XLNetForSequenceClassification.from_pretrained(best_checkpoint_dir)

最佳模型路径：/tmp/checkpoint_tmp_cde45623d157481e8b5d7ff7e1365b0b


In [15]:
trainer = Trainer(
    model=best_model,
    args=TrainingArguments(output_dir="./tmp"),  # 临时目录，仅用于预测
)

predictions = trainer.predict(test_dataset)
predictions_logits = predictions.predictions
predicted_labels = np.argmax(predictions_logits, axis=1)

accuracy = accuracy_score(test_dataset['label'], predicted_labels)
f1 = f1_score(test_dataset['label'], predicted_labels, average="binary")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (Macro Average): {f1:.4f}")
wandb.finish()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Accuracy: 0.9963
F1 Score (Macro Average): 0.9963


test/loss,▁
test/model_preparation_time,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/global_step,▁
test/loss,0.01636
test/model_preparation_time,0.0023
test/runtime,91.119
test/samples_per_second,157.025
test/steps_per_second,19.634


In [16]:
wandb.finish()